In [1]:
import sqlite3
import pandas as pd
import plotly.express as px

# Função para carregar dados de uma tabela específica no banco
def carregar_dados(tabela):
    db_path = r'c:\Users\ester\Downloads\rota-etl\superstore.db'
    
    conn = sqlite3.connect(db_path)
    query = f"SELECT * FROM {tabela}"
    df = pd.read_sql_query(query, conn)
    conn.close()
    
    return df

# Carregar os dados da tabela 'fact_vendas'
df_vendas = carregar_dados('fact_vendas')

# Visualizar os primeiros registros
print(df_vendas.head())


         order_id           order_date            ship_date customer_id  \
0  CA-2011-130813  2011-01-07 00:00:00  2011-01-09 00:00:00   LS-172304   
1  CA-2011-148614  2011-01-21 00:00:00  2011-01-26 00:00:00   MV-174854   
2  CA-2011-118962  2011-08-05 00:00:00  2011-08-09 00:00:00   CS-121304   
3  CA-2011-146969  2011-09-29 00:00:00  2011-10-03 00:00:00   AP-109154   
4  CA-2011-117317  2011-10-19 00:00:00  2011-10-19 00:00:00   JF-154904   

        product_id  sales  profit       ship_mode       state  quantity  \
0  OFF-PA-10002005     19  9.3312    Second Class  California         3   
1  OFF-PA-10002893     19  9.2928  Standard Class  California         2   
2  OFF-PA-10000659     21  9.8418  Standard Class  California         3   
3  OFF-PA-10002105      6  3.1104  Standard Class  California         1   
4  OFF-PA-10004519     13  6.5856        Same Day  California         3   

   discount  shipping_cost  
0       0.0           4.37  
1       0.0           0.94  
2       0.0

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import plotly.express as px
import pandas as pd

# Adicionar uma coluna de valor total de venda
df_vendas['valor_total'] = df_vendas['quantity'] * df_vendas['profit']

# Preencher os valores ausentes (se houver)
df_vendas = df_vendas.dropna(subset=['quantity', 'sales', 'profit', 'discount', 'shipping_cost'])

# Criar a variável dependente (target) e independente (features)
X = df_vendas[['quantity', 'sales', 'profit', 'discount', 'shipping_cost']]  # Quantidade, vendas
y = df_vendas['valor_total']  # Valor total de vendas

# Dividir em conjunto de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Criar o modelo de regressão linear
modelo = LinearRegression()
modelo.fit(X_train, y_train)

# Fazer previsões
y_pred = modelo.predict(X_test)

# Calcular o erro quadrático médio (MSE)
mse = mean_squared_error(y_test, y_pred)
# Calcular o R²
r2 = r2_score(y_test, y_pred)

# Exibir métricas
print(f"Erro Quadrático Médio (MSE): {mse}")
print(f"Coeficiente de Determinação (R²): {r2}")

# Criar um gráfico para visualizar os resultados
resultados = pd.DataFrame({'y_test': y_test, 'y_pred': y_pred})

# Gráfico de comparação entre valores reais e previstos
fig = px.scatter(resultados, x='y_test', y='y_pred', 
                 labels={'y_test': 'Valores Reais', 'y_pred': 'Valores Previstos'}, 
                 title='Comparação entre Valores Reais e Previstos',
                 template='plotly_dark')

# Adicionar uma linha de 45 graus (indicando perfeição)
fig.add_shape(type='line', x0=min(resultados['y_test']), y0=min(resultados['y_test']),
              x1=max(resultados['y_test']), y1=max(resultados['y_test']),
              line=dict(color='red', width=2, dash='dash'))

fig.show()


Erro Quadrático Médio (MSE): 385838.858698054
Coeficiente de Determinação (R²): 0.800181648434666


Conclusão dos Resultados

A análise de regressão linear para prever o valor total das vendas com base nas variáveis quantidade de produtos vendidos, vendas e lucro mostra um desempenho relativamente bom, com os 
seguintes resultados:

Erro Quadrático Médio (MSE): 3858.85
O MSE representa a média dos quadrados das diferenças entre os valores reais e os valores previstos. Quanto menor esse valor, melhor é o desempenho do modelo. O valor encontrado indica que, em média, há um erro de cerca de 385.858 nas previsões de valor total de vendas, o que sugere um desempenho razoável, mas com margem para melhorias.

Coeficiente de Determinação (R²): 0.800
O valor de R² de 0.791 indica que aproximadamente 80% da variação no valor total das vendas pode ser explicada pelas variáveis independentes escolhidas (quantidade, vendas, lucro, disconto e costo de envio). Isso significa que o modelo possui um bom poder explicativo e está capturando a maior parte da relação entre as variáveis de entrada e a variável de saída.

Implicações
O modelo é eficiente, já que consegue explicar um 80% da variabilidade dos valores das vendas, mas ainda pode ser melhorado.
A presença de uma diferença considerável entre os valores reais e previstos, indicada pelo MSE, sugere que outros fatores além de quantidade, vendas e lucro podem estar influenciando significativamente o valor total das vendas. A incorporação de outras variáveis, modos de envio e segmentação de clientes, poderia potencialmente melhorar o desempenho do modelo.
A linha de 45 graus no gráfico, que representa uma previsão perfeita, mostra que o modelo está bem ajustado para muitas observações, mas ainda existem discrepâncias que podem ser otimizadas.
Em resumo, o modelo tem uma boa performance preditiva, mas há espaço para melhorias adicionais, como a inclusão de variáveis adicionais ou técnicas mais avançadas de modelagem.